# Fama-MacBeth regressions

The regression approach of @Fama1973 is widely used in empirical asset pricing studies. 
Researchers use the two-stage regression approach to estimate risk premiums in various markets, but predominately in the stock market. 
Essentially, the two-step Fama-MacBeth regressions exploit a linear relationship between expected returns and exposure to (priced) risk factors. 
The basic idea of the regression approach is to project asset returns on factor exposures or characteristics that resemble exposure to a risk factor in the cross-section in each time period. 
Then, in the second step, the estimates are then aggregated across time to test if a risk factor is priced. 
In principle, Fama-MacBeth regressions can be used in the same way as portfolio sorts introduced in previous chapters.
In this chapter, we present a simple implementation of @Fama1973 to introduce the concept of their regressions. We use individual stocks as test assets to estimate the risk premium associated with the three factors included in [Fama1993](https://doi.org/2329112) .

The Fama-MacBeth procedure is a simple two-step approach: 
The first step uses the exposures (characteristics) as explanatory variables in $T$ cross-sectional regressions, i.e.,
$$\begin{aligned}r_{i,t+1} = \alpha_i + \lambda^{M}_t \beta^M_{i,t}  + \lambda^{SMB}_t \beta^{SMB}_{i,t} + \lambda^{HML}_t \beta^{HML}_{i,t} + \epsilon_{i,t}\text{, for each t}.\end{aligned}$$ 
Here, we are interested in the compensation $\lambda^{f}_t$ for the exposure to each risk factor $\beta^{f}_{i,t}$ at each time point, i.e., the risk premium. Note the terminology: $\beta^{f}_{i,t}$ is a asset-specific characteristic, e.g., a factor exposure or an accounting variable. *If* there is a linear relationship between expected returns and the characteristic in a given month, we expect the regression coefficient to reflect the relationship, i.e., $\lambda_t^{f}\neq0$. 

In the second step, the time-series average $\frac{1}{T}\sum\limits_{t=1}^T \hat\lambda^{f}_t$ of the estimates $\hat\lambda^{f}_t$ can then be interpreted as the risk premium for the specific risk factor $f$. We follow [Zaffaroni2022]( http://dx.doi.org/10.2139/ssrn.4111428) and consider the standard cross-sectional regression to predict future returns. If the characteristics are replaced with time $t+1$ variables, the regression approach rather captures risk attributes. 

Before we move to the implementation, we want to highlight that the characteristics, e.g., $\hat\beta^{f}_{i}$, are typically estimated in a separate step before applying the actual Fama-MacBeth methodology. You can think of this as a *step 0*. You might thus worry that the errors of $\hat\beta^{f}_{i}$ impact the risk premiums' standard errors. Measurement error in $\hat\beta^{f}_{i}$ indeed affects the risk premium estimates, i.e., they lead to biased estimates. The literature provides adjustments for this bias [see, e.g.[Chen2015]( https://doi.org/10.1016/j.qref.2014.12.002), among others] but also shows that the bias goes to zero as $T \to \infty$. We refer to [Gagliardini2016](https://doi.org/10.3982/ECTA11069) for an in-depth discussion also covering the case of time-varying betas. Moreover, if you plan to use Fama-MacBeth regressions with individual stocks: [Hou2020](https://doi.org/10.1093/rfs/hhy131) advocates using weighed-least squares to estimate the coefficients such that they are not biased towards small firms. Without this adjustment, the high number of small firms would drive the coefficient estimates.

In [1]:
import pandas as pd
import sqlite3
from datetime import timedelta
# Read sqlite query results into a pandas DataFrame
tidy_finance = sqlite3.connect("D:/Tidy/tidyfinance.sqlite")
crsp_monthly = pd.read_sql_query("SELECT * from crsp_monthly", tidy_finance)
beta = pd.read_sql_query("SELECT * from beta", tidy_finance)
compustat = pd.read_sql_query("SELECT * from compustat", tidy_finance)

In [5]:
crsp_monthly['month']=pd.to_datetime(crsp_monthly['month'])
beta['month']=pd.to_datetime(beta['month'])

In [7]:
bm=compustat.assign(month = pd.to_datetime(compustat['datadate']) + timedelta(days=1) - pd.DateOffset(months=1)).merge(crsp_monthly,left_on=["gvkey", "month"],right_on=["gvkey", "month"],how='left').merge(beta,left_on=["permno", "month"],right_on=["permno", "month"],how='left')

In [10]:
import numpy as np
bm=bm.assign( bm = bm.be / bm.mktcap , log_mktcap = np.log(bm.mktcap) , beta = bm.beta_monthly, sorting_date = bm.month +pd.DateOffset(months=6))[['gvkey','bm' ,'log_mktcap' ,'beta' ,'sorting_date']]

In [27]:
data_fama_macbeth=crsp_monthly.merge(bm,left_on=["gvkey", "month"],right_on=["gvkey", "sorting_date"],how='left').sort_values(['permno','month'])

In [28]:
data_fama_macbeth=data_fama_macbeth.assign(bm = data_fama_macbeth.groupby(['permno']).bm.fillna(method='ffill'),beta = data_fama_macbeth.groupby(['permno']).beta.fillna(method='ffill'),log_mktcap = data_fama_macbeth.groupby(['permno']).log_mktcap.fillna(method='ffill'))

In [29]:
data_fama_macbeth=data_fama_macbeth.merge(crsp_monthly.assign(ret_excess_lead = crsp_monthly.ret, month = crsp_monthly.month - pd.DateOffset(months=1))[['permno', 'month', 'ret_excess_lead']],left_on=["permno", "month"],right_on=["permno", "month"],how='left')[['permno', 'month', 'ret_excess_lead', 'beta', 'log_mktcap', 'bm']].dropna()

## Cross-sectional regression

Next, we run the cross-sectional regressions with the characteristics as explanatory variables for each month.  
We regress the returns of the test assets at a particular time point on each asset's characteristics. 
By doing so, we get an estimate of the risk premiums $\hat\lambda^{F_f}_t$ for each point in time. 

In [120]:
import statsmodels.formula.api as smf
def reg2(df):
    if df.dropna().shape[0]>0:  
        FF=df.copy()
        rp=smf.ols(formula='ret_excess_lead ~ 1+beta + log_mktcap + bm', data=FF).fit().params
    else:
        rp=[0,0,0,0]
    return tuple(rp)

In [121]:
reg=data_fama_macbeth.groupby('month').apply(lambda x:reg2(x)).reset_index()

In [122]:
risk_premiums = reg.rename(columns = {reg.columns[1]:'risk_premium'})
risk_premiums['Intercept']=risk_premiums.risk_premium.apply(pd.Series)[0]
risk_premiums['beta']=risk_premiums.risk_premium.apply(pd.Series)[1]
risk_premiums['log_mktcap']=risk_premiums.risk_premium.apply(pd.Series)[2]
risk_premiums['bm']=risk_premiums.risk_premium.apply(pd.Series)[3]

## Time-series aggregation

Now that we have the risk premiums' estimates for each period, we can average across the time-series dimension to get the expected risk premium for each characteristic. Similarly, we manually create the t-test statistics for each regressor, which we can then compare to usual critical values of 1.96 or 2.576 for two-tailed significance tests.

In [127]:
risk_premiums=risk_premiums[['month','Intercept','beta','log_mktcap','bm']].set_index('month').stack().reset_index().rename(columns={'level_1':'term',0:'estimate'})

In [143]:
price_of_risk=risk_premiums.groupby(['term']).agg( mean =('estimate', lambda x:  x.mean()) ,t_statistic = ('estimate', lambda x:  x.mean()/(x.std()/np.sqrt(len(x)))) )

In [144]:
print(price_of_risk)

                mean  t_statistic
term                             
Intercept   0.017438     7.010207
beta        0.000055     0.051775
bm          0.000911     2.178332
log_mktcap -0.001202    -3.394242


It is common to adjust for autocorrelation when reporting standard errors of risk premiums. As in chapter 5, the typical procedure for this is computing @Newey1987 standard errors. We again recommend the data-driven approach of @Newey1994 using the `NeweyWest()` function, but note that you can enforce the typical 6 lag settings via `NeweyWest(., lag = 6, prewhite = FALSE)`. 

In [163]:
def regnw(df):
    if df.dropna().shape[0]>0:  
        FF=df.copy()
        rp=smf.ols(formula='estimate ~ 1', data=FF).fit(cov_type='HAC',cov_kwds={'maxlags':6}).bse
    else:
        rp=0
    return float(rp)

In [167]:
price_of_risk_newey_west=risk_premiums.groupby('term').apply(lambda x:regnw(x)).rename('std_nw').reset_index().merge(price_of_risk,left_on=['term'],right_on=['term'])

In [172]:
price_of_risk_newey_west.assign(t_statistic_newey_west = price_of_risk_newey_west['mean'] / price_of_risk_newey_west.std_nw)

,term,std_nw,mean,t_statistic,t_statistic_newey_west
0,Intercept,0.002773,0.017438,7.010207,6.289310
1,beta,0.001158,0.000055,0.051775,0.047679
2,bm,0.000490,0.000911,2.178332,1.859533
3,log_mktcap,0.000373,-0.001202,-3.394242,-3.223626


Finally, let us interpret the results. Stocks with higher book-to-market ratios earn higher expected future returns, which is in line with the value premium. The negative value for log market capitalization reflects the size premium for smaller stocks. Lastly, the negative value for CAPM betas as characteristics is in line with the well-established betting against beta anomalies, indicating that investors with borrowing constraints tilt their portfolio towards high beta stocks to replicate a levered market portfolio [@Frazzini2014].